In [1]:
from Bio import PDB
from Bio.PDB.Polypeptide import PPBuilder
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np
import os

In [2]:
# Get and parse all pdb files in a folder

def parsePdbFiles(dir_path):
    structures = []
    
    files = os.listdir(dir_path)
    pdb_files = [(f, os.path.join(dir_path, f)) for f in files if f.endswith(".pdb")]
    
    for pdb, pdb_path in pdb_files:
        parser = PDB.PDBParser()
        structure = parser.get_structure(pdb, pdb_path) 
        structures.append(structure)
        
    return structures

In [3]:
structures = parsePdbFiles("pdb_sample")
structures

/usr/local/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 13044.
  PDBConstructionWarning)
/usr/local/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 13244.
  PDBConstructionWarning)
/usr/local/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 13414.
  PDBConstructionWarning)
/usr/local/lib/python2.7/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 13754.
  PDBConstructionWarning)


[<Structure id=6c16.pdb>,
 <Structure id=1ubi.pdb>,
 <Structure id=1ubq.pdb>,
 <Structure id=1lyz.pdb>,
 <Structure id=6fig.pdb>]

In [4]:
def getProteinInfo(structure):
    ppb = PPBuilder()
    for pp in ppb.build_peptides(structure):
        sequence = pp.get_sequence()
        print "sequence: ", sequence

        analysed_seq = ProteinAnalysis(str(sequence))
        print "molecular weight: ", analysed_seq.molecular_weight()
        print "amino acid counts: ", analysed_seq.count_amino_acids()
        print "---"

In [5]:
getProteinInfo(structures[0])

sequence:  AMPSIKLQSSDGEIFEVDVEIAKQSVTIKTMLED
molecular weight:  3753.2525
amino acid counts:  {'A': 2, 'C': 0, 'E': 4, 'D': 3, 'G': 1, 'F': 1, 'I': 4, 'H': 0, 'K': 3, 'M': 2, 'L': 2, 'N': 0, 'Q': 2, 'P': 1, 'S': 4, 'R': 0, 'T': 2, 'W': 0, 'V': 3, 'Y': 0}
---
sequence:  DDPVPLPNVNAAILKKVIQWCTHHKDDPP
molecular weight:  3261.7067
amino acid counts:  {'A': 2, 'C': 1, 'E': 0, 'D': 4, 'G': 0, 'F': 0, 'I': 2, 'H': 2, 'K': 3, 'M': 0, 'L': 2, 'N': 2, 'Q': 1, 'P': 5, 'S': 0, 'R': 0, 'T': 1, 'W': 1, 'V': 3, 'Y': 0}
---
sequence:  TDDIPVWDQEFLKVDQGTLFELILAANYLDIKGLLDVTCKTVANMIKGKTPEEIRKTFN
molecular weight:  6713.6814
amino acid counts:  {'A': 3, 'C': 1, 'E': 4, 'D': 6, 'G': 3, 'F': 3, 'I': 5, 'H': 0, 'K': 6, 'M': 1, 'L': 7, 'N': 3, 'Q': 2, 'P': 2, 'S': 0, 'R': 1, 'T': 6, 'W': 1, 'V': 4, 'Y': 1}
---
sequence:  DFTEEEEAQVRK
molecular weight:  1480.5325
amino acid counts:  {'A': 1, 'C': 0, 'E': 4, 'D': 1, 'G': 0, 'F': 1, 'I': 0, 'H': 0, 'K': 1, 'M': 0, 'L': 0, 'N': 0, 'Q': 1, 'P': 0, 'S': 0, 'R': 1

### Aminoacid pair distance Matrix

In [6]:
def AminoacidPairDistanceMatrix(structure):
    aa_vectors = {} # aminoacid vector dictionary {aa_id: aa_carbon_coordinate} 

    aa_id = 0 # aminoacid id
    for model in structure:
        for chain in model:
            for residue in chain:
                try:
                    coords_CA = residue['CA'].coord # x,y,z coordinates of the C-alpha atom
                    aa_vectors[aa_id] = coords_CA
                except:
                    continue
                aa_id += 1

    aa_distance_matrix = [] # Aminoacid pair distance matrix
    for vec in aa_vectors.values():
        dist_vec = []
        for vec_ in aa_vectors.values():
            distance = np.linalg.norm(vec-vec_)
            dist_vec.append(distance)
        aa_distance_matrix.append(dist_vec)
        
    return np.array(aa_distance_matrix)

In [7]:
protein_matrix_dict = {}
for protein in structures:
    protein_matrix = AminoacidPairDistanceMatrix(protein)
    protein_matrix_dict[protein.id] = protein_matrix
    print protein.id, protein_matrix.shape

6c16.pdb (510, 510)
1ubi.pdb (76, 76)
1ubq.pdb (76, 76)
1lyz.pdb (129, 129)
6fig.pdb (775, 775)


## Dimensionality Reduction with Autoencoders

In [8]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

In [19]:
def StackedAutoencoderWithTiedWeights(X_train, input_size, n_hidden1, n_hidden2):
    n_inputs = input_size 
    n_hidden1 = n_hidden1
    n_hidden2 =  n_hidden2 # codings
    n_hidden3 = n_hidden1 
    n_outputs = n_inputs

    learning_rate = 0.01
    l2_reg = 0.001

    activation = tf.nn.elu
    regularizer = tf.contrib.layers.l2_regularizer(l2_reg)
    initializer = tf.contrib.layers.variance_scaling_initializer()

    X = tf.placeholder(tf.float32, shape=[None, n_inputs])
    keep_prob = tf.placeholder(tf.float32)

    weights1_init = initializer([n_inputs, n_hidden1])
    weights2_init = initializer([n_hidden1, n_hidden2])

    weights1 = tf.Variable(weights1_init, dtype=tf.float32, name="weights1") 
    weights2 = tf.Variable(weights2_init, dtype=tf.float32, name="weights2") 
    weights3 = tf.transpose(weights2, name="weights3") # tied weights
    weights4 = tf.transpose(weights1, name="weights4") # tied weights

    biases1 = tf.Variable(tf.zeros(n_hidden1), name="biases1")
    biases2 = tf.Variable(tf.zeros(n_hidden2), name="biases2")
    biases3 = tf.Variable(tf.zeros(n_hidden3), name="biases3")
    biases4 = tf.Variable(tf.zeros(n_outputs), name="biases4")

    hidden1 = activation(tf.matmul(X, weights1) + biases1)
    hidden2 = activation(tf.matmul(hidden1, weights2) + biases2)
    hidden3 = activation(tf.matmul(hidden2, weights3) + biases3)
    outputs = tf.matmul(hidden3, weights4) + biases4

    reconstruction_loss = tf.reduce_mean(tf.square(outputs - X))
    reg_loss = regularizer(weights1) + regularizer(weights2)

    loss = reconstruction_loss + reg_loss

    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

    init = tf.global_variables_initializer()
    
    n_epochs = 5
    
    with tf.Session() as sess: 
        init.run() 
        for epoch in range(n_epochs):
            sess.run(training_op, feed_dict={X: X_train})
                
        features = hidden2.eval(feed_dict= {X: X_train})
                
    return features

In [20]:
protein_m = protein_matrix_dict["1ubi.pdb"]
X_train = [np.array(protein_m).flatten()]

features = StackedAutoencoderWithTiedWeights(X_train, len(X_train[0]), 50, 25)

In [21]:
features

array([[432.985   ,  -1.      ,  -1.      ,  -1.      ,  -1.      ,
         -1.      ,  -1.      ,  -1.      ,  -1.      , 174.59323 ,
         23.438284, 580.71204 ,  -1.      ,  -1.      ,  -1.      ,
         -1.      ,  -1.      ,  -1.      ,  -1.      , 776.46625 ,
         -1.      ,  -1.      , 260.1924  , 618.3449  ,  -1.      ]],
      dtype=float32)

In [ ]:
for protein, matrix in protein_matrix_dict.items():
    X_train = [np.array(matrix).flatten()]
    input_size = len(X_train[0])
    
    features = StackedAutoencoderWithTiedWeights(X_train, input_size, 1000, 500)
    dr
    print protein, matrix.shape
    print len(features)
    print features, "\n --------------------------------------------------------"

In [ ]:
from PIL import Image
img = Image.fromarray(np.array(residue_distance_matrix), 'RGB')
img.save('my2.png')
img.show()